In [94]:
import yfinance as yf

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [95]:
from stock_list import stocklist

In [96]:
stocklist.keys()

dict_keys(['sharpe_stocks', 'drawdown_stocks', 'esg_stocks2', 'esg_stock_ndsv', 'pe_stocks', 'test'])

# Variaables:

In [97]:
from datetime import datetime
selected_stocks = stocklist['drawdown_stocks']
start_date = '2020-01-01' #training start date
end_date = '2021-01-01' #training end date
starting_train = datetime.strptime(start_date, '%Y-%m-%d')
ending_test = datetime.strptime(end_date, '%Y-%m-%d')
ending_train = starting_train + (ending_test - starting_train) / 2
starting_test = ending_train
print(starting_train, ending_train, starting_test, ending_test)

2020-01-01 00:00:00 2020-07-02 00:00:00 2020-07-02 00:00:00 2021-01-01 00:00:00


# Data Preparation

In [98]:
def get_data(stock, start_date, end_date):
    data = yf.download(stock + '.NS', start=start_date, end=end_date)
    return data

In [99]:
df = pd.DataFrame()
for stock in selected_stocks:
    data = get_data(stock, start_date, end_date)
    df[stock] = data['Close']


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******

In [100]:
esg_data = pd.read_json('esg_data.json')

In [101]:
esg_data.head()


,esg,esg_industry,industry,name,ticker
0,[],[],Mining (except Oil & Gas),20-Microns-Ltd,20MICRONS
1,"[[1622505600000, 14.44], [1625097600000, 14.46...","[[1622505600000, 13.83], [1625097600000, 12.9]...",Containers & Packaging Manufacturing,3M-INDIA-LIMITED,3MINDIA
2,"[[1633046400000, 36.21], [1635724800000, 35.68...","[[1633046400000, 19.94], [1635724800000, 20.25...",Brokerage & Capital Markets,5paisa-Capital-Limited,5PAISA
3,[],[],Brokerage & Capital Markets,A-K-CAPITAL-SERVICES-LIMITED,530499
4,[],[],"Architectural, Engineering, & Related Services",A2Z-MAINTENANCE-and-ENGINEERING-SERVICES-LIMITED,A2ZINFRA


In [102]:
esg_data = esg_data[esg_data['ticker'].isin(selected_stocks)]
esg_data = esg_data.set_index('ticker')
esg_data.head()
# drop the row with name Nabha-Power-Limited
esg_data = esg_data[esg_data['name'] != 'Nabha-Power-Limited']

In [103]:
ESG_score_df = pd.DataFrame()
for ticker in selected_stocks:
    # print(ticker)
    sum = 0
    n = 0
    for el in esg_data.loc[ticker]['esg']:
      n = n + 1
      sum = sum + el[1]

    ESG_score_df[ticker] = [sum/n]

ESG_score_df = ESG_score_df.T
ESG_score_df.columns = ['ESG_score']
ESG_score_df.head()   

,ESG_score
ALKYLAMINE,26.132917
TATAINVEST,55.452941
JSWHL,46.280000
CONCOR,38.377600
ICICIBANK,59.359200


In [104]:
esg_data[esg_data.index == 'LT']

,esg,esg_industry,industry,name
ticker,,,,
LT,"[[1619827200000, 89.93], [1622505600000, 93.2]...","[[1619827200000, 81.17], [1622505600000, 82.59...","Heavy & Civil Engineering Construction, Electr...",Larsen-and-Toubro


# Helper Functions

In [105]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def is_positive_definite(Sigma):
    return np.all(np.linalg.eigvals(Sigma) > 0)

def calculate_portfolio_weights(Sigma, mu, Esg_score, u, l):
    one=np.ones(len(mu))
    # check if mu, Esg_score, one are linearly independent
    if np.linalg.matrix_rank(np.array([mu, Esg_score, one])) < 3:
        print("mu, Esg_score, one are not linearly independent")
        return
    if is_positive_definite(Sigma):
        Sigma_inv = np.linalg.inv(Sigma)
        a = np.transpose(mu) @ Sigma_inv @ mu
        b =  np.transpose(mu)@ Sigma_inv @ Esg_score
        c = np.transpose(mu) @ Sigma_inv @ one
        d = np.transpose(Esg_score) @ Sigma_inv @ Esg_score
        e = np.transpose(Esg_score) @ Sigma_inv @ one
        f = np.transpose(one) @ Sigma_inv @ one
        C = np.array([[a, b, c], [b, d, e], [c, e, f]])

        detC=np.linalg.det(C)

        C_inv = np.linalg.inv(C)

        aa=Sigma_inv @ mu
        bb=Sigma_inv @ Esg_score
        cc=Sigma_inv @ one 

        x0 = (1/detC)*((b*e-c*d)*aa+(b*c-a*e)*bb+(a*d-b*b)*cc)
        d2 = (1/detC)*((d*f-e*e)*aa+(c*e-b*f)*bb+(b*e-c*d)*cc)
        d3=(1/detC)*((c*e-b*f)*aa+(a*f-c*c)*bb+(b*c-a*e)*cc)


        # print(x0)
        # print(d2)
        # print(d3)

        x = x0 + u * d2 + l * d3

        # Output x
        # print(x)
        print("Sum of weights: ", np.sum(x))
        return x
    else:
        print("Matrix is not positive definite:\n")


In [106]:
def get_returns_given_w(weights, returns_df):
    aligned_weights = [weights[stock] for stock in returns_df.columns]
    mu = returns_df.pct_change().apply(lambda x: np.log(1+x)).mean()
    cov_matrix = returns_df.pct_change().apply(lambda x: np.log(1+x)).cov()
    portfolio_return = 252*(mu @ aligned_weights)
    portfolio_stddev = np.sqrt(np.array(aligned_weights).T @ cov_matrix @ np.array(aligned_weights)*252)
    return portfolio_return, portfolio_stddev

In [107]:
def sharpe_ratio(returns, volatility, risk_free_rate=0.06):
    return (returns - risk_free_rate) / volatility

In [108]:
df.index

DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-06',
               '2020-01-07', '2020-01-08', '2020-01-09', '2020-01-10',
               '2020-01-13', '2020-01-14',
               ...
               '2020-12-17', '2020-12-18', '2020-12-21', '2020-12-22',
               '2020-12-23', '2020-12-24', '2020-12-28', '2020-12-29',
               '2020-12-30', '2020-12-31'],
              dtype='datetime64[ns]', name='Date', length=251, freq=None)

# Analysis

In [109]:
train_df = df.loc[starting_train:ending_train]
test_df = df.loc[starting_test:ending_test]

In [110]:
mu = train_df.pct_change().apply(lambda x: np.log(1+x)).mean()
Sigma = train_df.pct_change().apply(lambda x: np.log(1+x)).cov()
Esg_score = ESG_score_df['ESG_score']
weights = calculate_portfolio_weights(Sigma, mu, Esg_score, 0.2/252, 60)
# make weights into a array with index
weights = pd.Series(weights, index=train_df.columns)

Sum of weights:  0.9999999999999989


In [111]:
mu.max()

0.005525815530448062

In [112]:
print("Sum of absolute weights: ", np.sum(np.abs(weights)))

Sum of absolute weights:  3.4475873229353997


In [113]:
training = get_returns_given_w(weights, train_df)
testing = get_returns_given_w(weights, test_df)

# Market Analysis

In [114]:
NIFTY = yf.download('^NSEI', start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [115]:
NIFTY_training = NIFTY.loc[starting_train:ending_train]
NIFTY_test = NIFTY.loc[starting_test:ending_test]

mu_NIFTY = NIFTY_training['Close'].pct_change().apply(lambda x: np.log(1+x)).mean()
var_NIFTY = NIFTY_training['Close'].pct_change().apply(lambda x: np.log(1+x)).var()

print("Returns of NIFTY in training:", mu_NIFTY*252)
print("Stddev of NIFTY in training:", np.sqrt(252*var_NIFTY))


mu_NIFTY_test = NIFTY_test['Close'].pct_change().apply(lambda x: np.log(1+x)).mean()
var_NIFTY_test = NIFTY_test['Close'].pct_change().apply(lambda x: np.log(1+x)).var()

print("Returns of NIFTY in test:", mu_NIFTY_test*252)
print("Stddev of NIFTY in test:", np.sqrt(252*var_NIFTY_test))

Returns of NIFTY in training: -0.2944373999728336
Stddev of NIFTY in training: 0.4245184965544325
Returns of NIFTY in test: 0.5629318098134067
Stddev of NIFTY in test: 0.15595218142958003


# Comparision

In [116]:
# tabulate the training and testing data for NIFTY and df (returns, stddev, sharpes ratio, esg score)
data = {'Returns': [training[0], testing[0], mu_NIFTY*252, mu_NIFTY_test*252], 
        'Stddev': [training[1], testing[1], np.sqrt(252*var_NIFTY), np.sqrt(252*var_NIFTY_test)], 
        'Sharpe Ratio': [sharpe_ratio(training[0], training[1]), sharpe_ratio(testing[0], testing[1]), sharpe_ratio(mu_NIFTY*252, np.sqrt(252*var_NIFTY)), sharpe_ratio(mu_NIFTY_test*252, np.sqrt(252*var_NIFTY_test))],
        'ESG Score': [Esg_score@weights, Esg_score@weights, None, None]}

df_compare = pd.DataFrame(data, index=['Training', 'Testing', 'NIFTY Training', 'NIFTY Testing'])
df_compare

,Returns,Stddev,Sharpe Ratio,ESG Score
Training,0.200000,0.143406,0.976249,60.0
Testing,0.172242,0.185989,0.603487,60.0
NIFTY Training,-0.294437,0.424518,-0.834916,NaN
NIFTY Testing,0.562932,0.155952,3.224910,NaN
